<a href="https://colab.research.google.com/github/DimaMirana/DataAnalysisWithPython/blob/master/IMDB_review_test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#check version of tensor
import tensorflow as tf
print(tf.__version__)

2.3.0


In [ ]:
import tensorflow_datasets as tfds
import numpy as np

imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)
#print(imdb.keys()) #dict_keys(['test', 'train', 'unsupervised'])

In [ ]:
train_data, test_data = imdb['train'], imdb['test']

sentences = []
labels = []

# merge default train test for spliting it into 80/20 later
for s,l in train_data:
  sentences.append(str(s.numpy()))
  labels.append(l.numpy())
  
for s,l in test_data:
  sentences.append(str(s.numpy()))
  labels.append(l.numpy())
  
#convert str to np array
sentences_final = np.array(sentences) 
labels_final = np.array(labels)

#print(len(sentences_final)) #50000
#print(labels_final)

# x = max([len(v) for v in training_sentences]) #get maximum review lingth
# print(x) #13747
# print(training_sentences[13747])
# print(len(training_sentences[13747]))
# print(training_labels_final[13747])

In [ ]:
#print(np.unique(labels_final)) # [0 1]
#print(len(np.unique(np.hstack(sentences_final)))) #49582 no of unique words
# review_length = [len(i) for i in sentences_final] #get all the length of each review stack in an array
#print(np.max(review_length)) #max review length 13741
#print(np.min(review_length)) #min review length 35
#print(np.mean(review_length)) #avg review length 1371.00638
#print(round(np.std(length))) #standard deviation 995.0
#print(sentences_final[0]) #This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.

In [ ]:
training_size=50000
test_portion=.8
split = int(training_size * test_portion)
vocab_size = 50000
embedding_dim = 16
max_length = 13741
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"


In [ ]:
training_sentences = sentences[0:split]
testing_sentences = sentences[split:]
training_labels = labels[0:split]
testing_labels = labels[split:]

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [ ]:
# Need this block to get it to work with TensorFlow 2.x
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

#print(decode_review(training_padded[2])) #padded review in str
#print(training_sentences[2]) #raw review in str
#print(training_labels[2]) #label for that review
#print(testing_sentences[1])
#print(testing_labels[1])


In [ ]:
#hidden layer1->global avg pooling
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

In [ ]:
num_epochs = 30
batch_size = 10
history = model.fit(training_padded, training_labels, batch_size=batch_size, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose = 2)

Epoch 1/30
4000/4000 - 74s - loss: 0.6920 - accuracy: 0.5165 - val_loss: 0.6800 - val_accuracy: 0.7436
Epoch 2/30
4000/4000 - 74s - loss: 0.5572 - accuracy: 0.7329 - val_loss: 0.4310 - val_accuracy: 0.8346
Epoch 3/30
4000/4000 - 73s - loss: 0.3784 - accuracy: 0.8434 - val_loss: 0.4274 - val_accuracy: 0.7882
Epoch 4/30
4000/4000 - 73s - loss: 0.3154 - accuracy: 0.8708 - val_loss: 0.2984 - val_accuracy: 0.8831
Epoch 5/30
4000/4000 - 73s - loss: 0.2829 - accuracy: 0.8858 - val_loss: 0.2831 - val_accuracy: 0.8880
Epoch 6/30
4000/4000 - 73s - loss: 0.2622 - accuracy: 0.8961 - val_loss: 0.2691 - val_accuracy: 0.8933
Epoch 7/30
4000/4000 - 74s - loss: 0.2436 - accuracy: 0.9046 - val_loss: 0.2780 - val_accuracy: 0.8861
Epoch 8/30
4000/4000 - 73s - loss: 0.2286 - accuracy: 0.9105 - val_loss: 0.2534 - val_accuracy: 0.9013
Epoch 9/30
4000/4000 - 73s - loss: 0.2148 - accuracy: 0.9177 - val_loss: 0.2590 - val_accuracy: 0.8956
Epoch 10/30
4000/4000 - 76s - loss: 0.2032 - accuracy: 0.9208 - val_loss:

In [ ]:
num_epochs = 50
batch_size = 8
history6 = model.fit(training_padded, training_labels, batch_size=batch_size, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose = 2)

Epoch 1/50
5000/5000 - 79s - loss: 0.6904 - accuracy: 0.5269 - val_loss: 0.6710 - val_accuracy: 0.6144
Epoch 2/50
5000/5000 - 80s - loss: 0.5514 - accuracy: 0.7394 - val_loss: 0.4383 - val_accuracy: 0.8210
Epoch 3/50
5000/5000 - 80s - loss: 0.3808 - accuracy: 0.8404 - val_loss: 0.3354 - val_accuracy: 0.8683
Epoch 4/50
5000/5000 - 80s - loss: 0.3141 - accuracy: 0.8733 - val_loss: 0.2976 - val_accuracy: 0.8834
Epoch 5/50
5000/5000 - 80s - loss: 0.2824 - accuracy: 0.8864 - val_loss: 0.2795 - val_accuracy: 0.8884
Epoch 6/50
5000/5000 - 81s - loss: 0.2621 - accuracy: 0.8950 - val_loss: 0.3348 - val_accuracy: 0.8554
Epoch 7/50
5000/5000 - 81s - loss: 0.2422 - accuracy: 0.9042 - val_loss: 0.2565 - val_accuracy: 0.8982
Epoch 8/50
5000/5000 - 80s - loss: 0.2301 - accuracy: 0.9096 - val_loss: 0.2611 - val_accuracy: 0.8939
Epoch 9/50
5000/5000 - 80s - loss: 0.2162 - accuracy: 0.9160 - val_loss: 0.2491 - val_accuracy: 0.9030
Epoch 10/50
5000/5000 - 80s - loss: 0.2046 - accuracy: 0.9204 - val_loss:

In [ ]:
#hidden layer1->global avg pooling
model2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(32, kernel_initializer='lecun_normal', activation='selu'),
    tf.keras.layers.AlphaDropout(0.01),
    tf.keras.layers.Dense(16, kernel_initializer='lecun_normal', activation='selu'),
    tf.keras.layers.AlphaDropout(0.01),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model2.summary()

In [ ]:
num_epochs = 20
batch_size = 10
history2 = model2.fit(training_padded, training_labels, batch_size=batch_size, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose = 2) #not good model, accuracy 40-50 flactuate

Epoch 1/20
4000/4000 - 72s - loss: 0.6976 - accuracy: 0.5039 - val_loss: 0.6898 - val_accuracy: 0.5110
Epoch 2/20
4000/4000 - 73s - loss: 0.5866 - accuracy: 0.6729 - val_loss: 0.3999 - val_accuracy: 0.8318
Epoch 3/20
4000/4000 - 73s - loss: 0.3941 - accuracy: 0.8223 - val_loss: 0.3417 - val_accuracy: 0.8538
Epoch 4/20
4000/4000 - 74s - loss: 0.3257 - accuracy: 0.8616 - val_loss: 0.3179 - val_accuracy: 0.8641
Epoch 5/20
4000/4000 - 73s - loss: 0.2892 - accuracy: 0.8824 - val_loss: 0.2772 - val_accuracy: 0.8895
Epoch 6/20
4000/4000 - 74s - loss: 0.2671 - accuracy: 0.8934 - val_loss: 0.2627 - val_accuracy: 0.8947
Epoch 7/20
4000/4000 - 74s - loss: 0.2425 - accuracy: 0.9046 - val_loss: 0.2703 - val_accuracy: 0.8918
Epoch 8/20
4000/4000 - 74s - loss: 0.2291 - accuracy: 0.9100 - val_loss: 0.2561 - val_accuracy: 0.8994
Epoch 9/20
4000/4000 - 74s - loss: 0.2154 - accuracy: 0.9166 - val_loss: 0.2523 - val_accuracy: 0.9023
Epoch 10/20
4000/4000 - 75s - loss: 0.2033 - accuracy: 0.9221 - val_loss:

In [ ]:
# hidden layer 2 -> global avg poolinh
model5 = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(50,activation = 'relu'),
        tf.keras.layers.Dense(25,activation = 'relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(12,activation = 'relu'),
        f.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1,activation = 'sigmoid'),
])
model5.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
model5.summary()

In [ ]:
num_epochs = 30
batch_size = 10
history5 = model5.fit(training_padded, training_labels, batch_size=batch_size, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose = 2)

Epoch 1/30
4000/4000 - 76s - loss: 0.6932 - accuracy: 0.4983 - val_loss: 0.6931 - val_accuracy: 0.5030
Epoch 2/30
4000/4000 - 76s - loss: 0.6932 - accuracy: 0.4981 - val_loss: 0.6932 - val_accuracy: 0.4970
Epoch 3/30
4000/4000 - 75s - loss: 0.6932 - accuracy: 0.4964 - val_loss: 0.6931 - val_accuracy: 0.5030
Epoch 4/30
4000/4000 - 75s - loss: 0.6932 - accuracy: 0.4985 - val_loss: 0.6932 - val_accuracy: 0.4970
Epoch 5/30
4000/4000 - 76s - loss: 0.6932 - accuracy: 0.4956 - val_loss: 0.6931 - val_accuracy: 0.5030
Epoch 6/30
4000/4000 - 75s - loss: 0.6932 - accuracy: 0.4993 - val_loss: 0.6932 - val_accuracy: 0.4970
Epoch 7/30
4000/4000 - 75s - loss: 0.6932 - accuracy: 0.4992 - val_loss: 0.6932 - val_accuracy: 0.4970
Epoch 8/30
4000/4000 - 76s - loss: 0.6932 - accuracy: 0.5006 - val_loss: 0.6931 - val_accuracy: 0.5030
Epoch 9/30
4000/4000 - 78s - loss: 0.6931 - accuracy: 0.5041 - val_loss: 0.6937 - val_accuracy: 0.4970
Epoch 10/30
4000/4000 - 76s - loss: 0.6932 - accuracy: 0.4983 - val_loss:

In [ ]:
#lstm
# model3 = tf.keras.Sequential([
#     tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
#     tf.keras.layers.LSTM(32),
#     tf.keras.layers.Dense(16, activation='relu'),
#     tf.keras.layers.Dense(1, activation='sigmoid')
# ])
model3 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model3.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model3.summary()

In [ ]:
num_epochs = 3
batch_size = 10
history3 = model3.fit(training_padded, training_labels, batch_size=batch_size, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose = 1)

Epoch 1/3
  16/4000 [..............................] - ETA: 33:15:17 - loss: 0.6940 - accuracy: 0.4750

In [ ]:
max_features = 6000
embed_size = 80
model7 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embed_size),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences = True)),
    tf.keras.layers.GlobalMaxPool1D(),
    tf.keras.layers.Dense(20,activation='relu'),
    tf.keras.layers.Dropout(0.05),
    tf.keras.layers.Dense(1,activation='sigmoid')
])
model7.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model7.summary()


In [ ]:
batch_size = 10
epochs = 3
history7 = model7.fit(training_padded, training_labels, batch_size=batch_size, epochs=epochs, validation_data=(testing_padded, testing_labels), verbose = 2)

In [ ]:
#mix of conv & lstm
model4 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model4.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model4.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 13741, 16)         800000    
_________________________________________________________________
dropout_11 (Dropout)         (None, 13741, 16)         0         
_________________________________________________________________
conv1d (Conv1D)              (None, 13737, 64)         5184      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 3434, 64)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_43 (Dense)             (None, 1)                 65        
Total params: 838,273
Trainable params: 838,273
Non-trainable params: 0
_______________________________________________

In [ ]:
num_epochs = 30
batch_size = 10
history4 = model4.fit(training_padded, training_labels, batch_size=batch_size, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose = 2)

In [ ]:
model8 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model8.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model8.summary()

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

plot_graphs(history, 'accuracy')
plot_graphs(history, 'loss')
plot_graphs(history6, 'accuracy')
plot_graphs(history6, 'loss')
plot_graphs(history2, 'accuracy')
plot_graphs(history2, 'loss')
plot_graphs(history5, 'accuracy')
plot_graphs(history5, 'loss')
# plot_graphs(history3, 'accuracy')
# plot_graphs(history3, 'loss')
# plot_graphs(history4, 'accuracy')
# plot_graphs(history4, 'loss')